# Explore the input data and build a model

In [ ]:
import copy
import cPickle as pickle
from datetime import datetime
from IPython.display import display
import logging
from matplotlib.dates import MonthLocator, WeekdayLocator, DateFormatter
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
import pandas as pd
import psycopg2
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc, classification_report, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
import sys
import time

plt.style.use('ggplot')
%matplotlib inline

In [ ]:
# Show all columns and rows.
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Set up logging

In [ ]:
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(process)d/%(threadName)s - %(name)s - %(levelname)s - %(message)s',
                    stream=sys.stdout)
logger = logging.getLogger('main()')

In [ ]:
epoch = int(time.time())
epoch

In [ ]:
str_file_csv = 'historical_data.csv'
df_csv = pd.read_csv('../../../data/input/' + str_file_csv,
                     parse_dates=['created_at',
                                  'actual_delivery_time'])
df_csv.head()

In [ ]:
df_csv.shape

### Look at histograms

In [ ]:
df_csv['market_id'].value_counts().sort_index()

In [ ]:
df_csv['created_at'].dt.hour.value_counts().sort_index()

In [ ]:
# Monday == 0.
df_csv['created_at'].dt.dayofweek.value_counts().sort_index()

In [ ]:
df_csv['store_id'].value_counts().sort_index()

In [ ]:
df_csv['store_primary_category'].value_counts().sort_index()

In [ ]:
df_csv['order_protocol'].value_counts().sort_index()

### Make sure that datetimes are parsed correctly

In [ ]:
type(df_csv['created_at'][0])

In [ ]:
type(df_csv['actual_delivery_time'][0])

### Look for NaNs

In [ ]:
df_tmp = df_csv.isnull().any()
df_tmp[ df_tmp==True ]

## Drop rows where the outcome variable cannot be computed.
"The target value to predict here is the total seconds value between `created_at` and `actual_delivery_time`"

In [ ]:
df_csv = df_csv[ ~df_csv['created_at'].isnull() ]
df_csv = df_csv[ ~df_csv['actual_delivery_time'].isnull() ]

In [ ]:
df_csv.shape

### Look for outliers

In [ ]:
cols_boxplot = df_csv.columns.values.tolist()

cols_boxplot.remove('created_at')
cols_boxplot.remove('actual_delivery_time')
cols_boxplot.remove('store_id')
cols_boxplot.remove('store_primary_category')

In [ ]:
for col in cols_boxplot:
    plt.figure()
    df_csv.boxplot(column=col)

In [ ]:
## Generate features

In [ ]:
### Encode the hour and the day of the week of the creation datetime

In [ ]:
df_csv['created_at_hour'] = df_csv['created_at'].dt.hour
df_csv['created_at_dayofweek'] = df_csv['created_at'].dt.dayofweek

### Fill NaNs
* Use median if the column is continuous
* Use mode if the column is categorical
* Keep track of the medians and mode of all numerical columns because we will use those numbers to fill NaNs in production
* The set of features should be `cols_categorical` + `cols_cont` + one-hot-encoded `store_primary_category`

In [ ]:
cols_categorical = ['market_id',
                    'order_protocol']

In [ ]:
df_csv[cols_categorical].mode()

In [ ]:
cols_cont = ['created_at_hour',
             'created_at_dayofweek',
             'total_items',
              'subtotal',
              'num_distinct_items',
              'min_item_price',
              'max_item_price',
              'total_onshift_dashers',
              'total_busy_dashers',
              'total_outstanding_orders',
              'estimated_order_place_duration',
              'estimated_store_to_consumer_driving_duration']

In [ ]:
df_csv[cols_cont].median()

In [ ]:
for col in cols_categorical:
    df_csv[col].fillna(df_csv[col].mode()[0], inplace=True) 

In [ ]:
for col in cols_cont:
    df_csv[col].fillna(df_csv[col].median(), inplace=True) 

In [ ]:
df_csv[['store_primary_category']] = df_csv[['store_primary_category']].fillna(value='unknown')

In [ ]:
# Check for NaNs again.
df_tmp = df_csv.isnull().any()
df_tmp[ df_tmp==True ]

### TODO: One-hot-encode certain categorical columns

In [ ]:
cols_features = cols_categorical + cols_cont
cols_features

### Compute outcome variable

In [ ]:
col_outcome = 'outcome_total_delivery_time'
df_csv[col_outcome] = ( df_csv['actual_delivery_time'] - df_csv['created_at'] ) / np.timedelta64(1, 's')
df_csv.head()

In [ ]:
df_csv.boxplot(column=[col_outcome])

### Replace outliers in the outcome variable with the median

In [ ]:
df_csv[col_outcome].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.90, 0.95, 0.99])

In [ ]:
df_csv.loc[ df_csv[col_outcome] > 1e4, col_outcome] = np.nan
df_csv[col_outcome].fillna(df_csv[col_outcome].median(), inplace=True)

In [ ]:
df_csv.boxplot(column=[col_outcome])

## Split dataset into training and test

In [ ]:
# Write the cleaned dataset to file.
df_csv.to_csv('df_csv' + str(epoch) + '.csv', index=False)

In [ ]:
df_train, df_test = train_test_split(df_csv, test_size=0.1, random_state=5)

In [ ]:
df_train.shape

In [ ]:
df_test.shape

### Train and pickle the model

In [ ]:
X_train = df_train[cols_features]
X_train.shape

In [ ]:
X_test = df_test[cols_features]
X_test.shape

In [ ]:
y_train = df_train[col_outcome]
y_test = df_test[col_outcome]

In [ ]:
rf = RandomForestRegressor(n_estimators=100, n_jobs=1)
rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
# Pickle the model
dir_out = '/media/sf_Documents/'
with open(dir_out + 'rf.' + str(epoch) + '.pkl', 'wb') as f:
    pickle.dump(rf, f)

### Plot the results

In [ ]:
f, (ax1, ax2) = plt.subplots(2, sharex=True)
ax1.plot( y_pred, y_test, '.' )
ax1.set_ylabel('Observed Total Delivery\nTime (seconds)')

ax2.plot( y_pred, y_test - y_pred, '.' )
ax2.set_xlabel('Predicted Total Delivery Time (seconds)')
ax2.set_ylabel('Residual\n(seconds)')
plt.show()

In [ ]:
RMSE = np.sqrt( mean_squared_error(y_test, y_pred) )
RMSE